In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import h5py
import os, sys, traceback
from synthgen import *
from common import *
import pickle
from glob import glob
import os
import math
from PIL import Image
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

В цьому ноутбуці ви знадете код для генерації синтетичних картинок з текстом, як описано в статті ["Synthetic Data for Text Localisation in Natural Images"](http://www.robots.ox.ac.uk/~vgg/data/scenetext/) з деякими модифікаціями: доданий код для генерації обрізаних текстових bounding boxes спеціально для задачі text recognition, так званих кропів. Також додана можливість генерувати не лише англійський текст, а і будь-якою іншою мовою. Для цього просто створіть свій текстовий файл, помістьть туди який побажаєте корпус і вкажіть шлях до файлу в параметр `TEXT_CORPUS_FILE`. Ми рекомендуємо зробити певний препроцесінг для тексту: видалити емоджі, "погані символи" і т д.
Також майже все тут зберігається в h5 файлах. Що це і як з ними правильно працювати можна почитати [тут](http://docs.h5py.org/en/stable/build.html)

In [ ]:
NUM_IMG = -1 #number of images to use for generation (-1 to use all available):
INSTANCE_PER_IMAGE = 10 #number of times to use the same image
SECS_PER_IMG = 5 # max time per image in seconds
TEXT_CORPUS_FILE = 'data/newsgroup/newsgroup.txt' # file with text corpus to use in generation
PATH_TO_IMNAMES = 'imnames.cp' #file with background image names
IMAGES_DIRECTORY = 'bg_img/' #directory with background images 
PATH_TO_DEPTH_DATABASE = 'depth.h5' #file with depth for each background image
PATH_TO_SEG_DATABASE = 'seg.h5' #file with segmentation for each background image
SAVE_IMAGES = False #set True if you want to store not only crops, but also whole images for text detection task (or end-to-end)
SYNTH_IMS_DIR = 'results/' #where to store synthetic images if SAVE_IMAGES 
prefix_for_results = 'synth_text_result_' #this prefix will be used for each batch of images if SAVE_IMAGES
SAVE_CROPS = True # set True if you want to store crops for text recognition task
CROPS_DIR = 'crops_sample/' #where to store crops if SAVE_CROPS 
viz = False #set True if you want to visualize each synththetic image


num_threads = 10 #
batch_size = 50 # how much background images use to store data in one h5 base (for backup)


if SAVE_CROPS and not os.path.exists(CROPS_DIR):
    os.mkdir(CROPS_DIR)
if SAVE_IMAGES and not os.path.exists(SYNTH_IMS_DIR):
    os.mkdir(SYNTH_IMS_DIR)

In [ ]:
depth_db = h5py.File(PATH_TO_DEPTH_DATABASE,'r')
seg_db = h5py.File(PATH_TO_SEG_DATABASE, 'r')['mask']
imnames = pickle.load(open(PATH_TO_IMNAMES, 'rb'))
images = [os.path.basename(x) for x in glob(f"{IMAGES_DIRECTORY}/*jpg")]
imnames = list(filter(lambda x: x in images, imnames))  

In [ ]:
def add_res_to_db(imgname,res,db):
    """
    Add the synthetically generated text image instance
    and other metadata to the dataset.
    """
    ninstance = len(res)
    for i in range(ninstance):
        dname = "%s_%d"%(imgname, i)
        db['data'].create_dataset(dname,data=res[i]['img'])
        db['data'][dname].attrs['charBB'] = res[i]['charBB']
        db['data'][dname].attrs['wordBB'] = res[i]['wordBB']  
        L = res[i]['txt']
        L = [n.encode("utf-8") for n in L]
        db['data'][dname].attrs['txt'] = L

Це код для того, щоб отримати рівний, правильно повернутий кроп з bounding box. Інколи він може видавати помилки(наприклад, коли текст занадто близько до краю), а іноді - неправильно повертати текст і віддавати перевернутий. В продакшині в нас працює певний воркераунд: ми віддаємо дві картинки, ту, яку віддав наш метод і повернуту на 180 градусів, але це не надто хороше рішення, якщо ми хочемо обробляти набагато більший потік картинок. Тому feel free до будь-яких модифікацій.

In [ ]:
def get_crop(pil_image, poly):
    rotate_180 = None
    img = np.array(pil_image)[:, :, ::-1]
    polygon = [poly['x0'], poly['y0'], poly['x1'], poly['y1'], poly['x2'], poly['y2'], poly['x3'], poly['y3']]
    height = math.sqrt((polygon[6] - polygon[0]) ** 2 + (polygon[7] - polygon[1]) ** 2)
    width = math.sqrt((polygon[2] - polygon[0]) ** 2 + (polygon[3] - polygon[1]) ** 2)
    try:
        angle = math.pi - math.asin((polygon[7] - polygon[5]) / width)
        rotate_180 = True
    except:
        cr = pil_image.crop((min(polygon[0], polygon[6]),
                             min(polygon[1], polygon[3]),
                             max(polygon[4], polygon[2]),
                             max(polygon[5], polygon[7]))).convert('RGB')
        try:
            z = np.array(cr)[:, :, ::-1]
        except:
            return None
    else:
        x_center = (polygon[0] + polygon[4]) / 2
        y_center = (polygon[1] + polygon[5]) / 2
        rect = ((x_center, y_center), (width, height), angle * 180 / math.pi)
        z = crop_minAreaRect(img, rect)
        if 0 in z.shape:
            angle = math.asin((polygon[3] - polygon[5]) / height) + math.pi / 2
            rect = ((x_center, y_center), (width, height), angle * 180 / math.pi)
            z = crop_minAreaRect(img, rect)
            rotate_180 = False
    if z.shape[0] / z.shape[1] > 0.8:
        z = np.rot90(z, 3)
    if rotate_180:
        z = np.rot90(z, 2)
    return z


def crop_minAreaRect(img, rect):
    # rotate img
    angle = rect[2]
    rows, cols = img.shape[0], img.shape[1]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)

    c = round((rows ** 2 + cols ** 2) ** 0.5)
    img_rot = cv2.warpAffine(img, M, (c, c))
    box = cv2.boxPoints(rect)
    pts = np.int0(cv2.transform(np.array([box]), M))[0]
    pts[pts < 0] = 0
    # crop
    img_crop = img_rot[pts[1][1]:pts[0][1],
               pts[1][0]:pts[2][0]]

    return img_crop

А тут знаходиться код для самого вирізання кропів і зберігання розмітки. Як ви можете побачити, ми встановили обмеження на мінімальну довжину тексту 4. Ви можете прибрати чи змінити його, додати нові або не використовувати обмежень взагалі і подати дужееее правильний корпус

In [ ]:
def generate_crops(res):
    num_images_exists = len(glob(f"{CROPS_DIR}/*jpg")) 
    ninstance = len(res)
    for k in range(ninstance):
        rgb = res[k]['img']
        wordBB = res[k]['wordBB']
        txt = res[k]['txt']

        res_txt = []
        for word in txt:
            if len(word.split()) > 1:
                res_txt.extend(word.split())
            else:
                res_txt.append(word)
        txt = res_txt
        image = Image.fromarray(rgb)
        
        for i in range(wordBB.shape[-1]):
            bb = wordBB[:,:,i]
            bb = np.c_[bb,bb[:,0]]
            image = Image.fromarray(rgb)
            poly = {
            'x0': bb[0, 0],
            'x1': bb[0, 1],
            'x2': bb[0, 2],
            'x3': bb[0, 3],
            'y0': bb[1, 0],
            'y1': bb[1, 1],
            'y2': bb[1, 2],
            'y3': bb[1, 3],
            }
            label = txt[i]
            try:
                img_cropped = get_crop(image, poly)
                crop_pil = Image.fromarray(img_cropped)
            except Exception as e:
                print(e)
                continue
            if len(label) >=4:
                name = f"{CROPS_DIR}/{num_images_exists}.jpg"
                crop_pil.save(name)
                with open(name.replace('jpg', 'txt'), 'w', encoding='utf-8') as annotation:
                    annotation.write(label + '\n')
                num_images_exists += 1

In [ ]:
def viz_textbb(text_im, charBB_list, wordBB, alpha=1.0):
    """
    text_im : image containing text
    charBB_list : list of 2x4xn_i bounding-box matrices
    wordBB : 2x4xm matrix of word coordinates
    """
    plt.figure(figsize=(20, 20))
    plt.imshow(text_im)

    H,W = text_im.shape[:2]

    # plot the character-BB:
    for i in range(len(charBB_list)):
        bbs = charBB_list[i]
        ni = bbs.shape[-1]
        for j in range(ni):
            bb = bbs[:,:,j]
            bb = np.c_[bb,bb[:,0]]
            plt.plot(bb[0,:], bb[1,:], 'r', alpha=alpha/2)

    # plot the word-BB:
    for i in range(wordBB.shape[-1]):
        bb = wordBB[:,:,i]
        bb = np.c_[bb,bb[:,0]]
        plt.plot(bb[0,:], bb[1,:], 'g', alpha=alpha)
        # visualize the indiv vertices:
        vcol = ['r','g','b','k']
        for j in range(4):
            plt.scatter(bb[0,j],bb[1,j],color=vcol[j])        

    plt.show()

In [ ]:
global_batch = num_threads * batch_size


def preprocess_batch(name_and_batch):
    name_db, batch = name_and_batch
    if SAVE_IMAGES:
        out_db = h5py.File(name_db,'w')
        out_db.create_group('/data')
    for imname in batch:
        try:
            img = Image.open(os.path.join(IMAGES_DIRECTORY, imname))
            
            depth = depth_db[imname][:].T
            depth = depth[:,:,1]
            seg = seg_db[imname][:].astype('float32')
            area = seg_db[imname].attrs['area']
            label = seg_db[imname].attrs['label']

            sz = depth.shape[:2][::-1]
            img = np.array(img.resize(sz,Image.ANTIALIAS))
            seg = np.array(Image.fromarray(seg).resize(sz,Image.NEAREST))
            res = RV3.render_text(img,depth,seg,area,label,
                                ninstance=INSTANCE_PER_IMAGE,viz=False)
            if viz:
                 for k in range(len(res)):
                    rgb = res[k]['img']
                    charBB = res[k]['charBB']
                    wordBB = res[k]['wordBB']
                    txt = res[k]['txt']
                    viz_textbb(rgb, [charBB], wordBB)
            if len(res) > 0:
                if SAVE_CROPS:
                    generate_crops(res)
                if SAVE_IMAGES:
                    add_res_to_db(imname,res,out_db)
        except Exception as e:
            continue
    if SAVE_IMAGES:
        out_db.close()

А це власне код для генерації картинок. Це може зайняти певний час - кілька годин

In [ ]:
N = len(imnames)
if NUM_IMG < 0:
    NUM_IMG = N
RV3 = RendererV3('./data/',TEXT_CORPUS_FILE, max_time=SECS_PER_IMG)

for i in tqdm(range(min(NUM_IMG, N) // global_batch)):
    names_global_batch = imnames[i * global_batch: (i + 1) * global_batch]
    batches = []
    names = []
    for j in range(num_threads):
        batches.append(imnames[j * batch_size: (j + 1) * batch_size])
        names.append(f'{SYNTH_IMS_DIR}/{prefix_for_results}{i}_{j}.h5')
    list_to_preprocess = list(zip(names, batches))
    pool = Pool(num_threads)
    pool.map(preprocess_batch, list_to_preprocess)